In [201]:
from llama_cpp import Llama
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [202]:
# model_path="C:/Users/benjc/Documents/models/qwen2-0_5b-instruct-fp16.gguf"
model_path = "C:/Users/benjc/Documents/models/Qwen2-7B-Instruct-Q4_K_M.gguf"

In [203]:
system_prompt = """
Given the sentence, generate at least twenty paraphrased versions that maintain the original semantic meaning and style. 
Return the rephrased sentences in a Python list format, ensuring the output starts with '[' and ends with ']'. 
Include no additional text or notes in the output.
"""

examples = [
    {"sentence": "DBC Pierre, Booker Prize-winner and author of Vernon God Little, has moved to Ireland.",
     "rephrased_sentences": ["Booker Prize-winner DBC Pierre, known for writing Vernon God Little, has moved to Ireland.", 
                             "The author of Vernon God Little and Booker Prize-winner, DBC Pierre, has settled in Ireland.", 
                             "DBC Pierre, who won the Booker Prize for Vernon God Little, has now moved to Ireland.", 
                             "DBC Pierre, celebrated for his Booker Prize-winning novel Vernon God Little, has moved to Ireland.", 
                             "Winner of the Booker Prize and author of Vernon God Little, DBC Pierre, has relocated to Ireland.", 
                             "The Booker Prize-winning author of Vernon God Little, DBC Pierre, has taken up residence in Ireland.", 
                             "DBC Pierre, the acclaimed Booker Prize-winner for Vernon God Little, has moved to Ireland.", 
                             "Booker Prize laureate and Vernon God Little's author, DBC Pierre, has moved to Ireland.", 
                             "DBC Pierre, the author of the Booker Prize-winning novel Vernon God Little, has moved to Ireland.", 
                             "DBC Pierre, recognized for his Booker Prize win with Vernon God Little, has moved to Ireland.", 
                             "DBC Pierre, famous for Vernon God Little and a Booker Prize-winner, has moved to Ireland.", 
                             "Booker Prize-winning author DBC Pierre, known for Vernon God Little, has now moved to Ireland.", 
                             "DBC Pierre, who won the Booker Prize for his novel Vernon God Little, has relocated to Ireland.", 
                             "The author DBC Pierre, who won the Booker Prize for Vernon God Little, has moved to Ireland.", 
                             "DBC Pierre, who penned the Booker Prize-winning Vernon God Little, has moved to Ireland.", 
                             "DBC Pierre, a Booker Prize winner for Vernon God Little, has moved to Ireland.", 
                             "DBC Pierre, recognized for his Booker Prize-winning book Vernon God Little, has moved to Ireland.", 
                             "Booker Prize-winning author DBC Pierre, famous for Vernon God Little, has moved to Ireland.", 
                             "DBC Pierre, the Booker Prize-winning novelist of Vernon God Little, has moved to Ireland."]
    },
    {"sentence": "Known for being very delicate, the skill could take a lifetime to master.",
     "rephrased_sentences": ["Famous for being delicate, this skill might require a lifetime to master.", 
                             "This skill, known for its delicacy, could take a lifetime to master.", 
                             "The skill's reputation for delicacy means it could take a lifetime to master.", 
                             "Mastering this skill, which is known for its delicacy, might take a lifetime.", 
                             "The delicacy of this skill suggests it could take a lifetime to master.", 
                             "Renowned for its delicateness, the skill might take a lifetime to master.", 
                             "Due to its delicate nature, the skill could take a lifetime to master.", 
                             "The skill, famed for its delicacy, might require a lifetime to master.", 
                             "Known for its delicate nature, this skill could take a lifetime to master.", 
                             "Because of its delicateness, the skill could take a lifetime to master.", 
                             "The skill, recognized for its delicacy, could take a lifetime to master.", 
                             "This skill, due to its delicate nature, might take a lifetime to master.", 
                             "The skill, famous for being very delicate, could take a lifetime to master.", 
                             "The delicate nature of the skill means it might take a lifetime to master.", 
                             "Given its delicacy, the skill could take a lifetime to master.", 
                             "The skill's delicate nature suggests it could take a lifetime to master.", 
                             "Known for its very delicate nature, the skill could take a lifetime to master.", 
                             "Mastering this delicate skill could require a lifetime.", 
                             "This skill, noted for its delicacy, might take a lifetime to master."]
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{sentence}"),
        ("ai", "{rephrased_sentences}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        few_shot_prompt,
        ("human", "{sentence}"),
    ]
)

In [204]:
llm = Llama(
    model_path=model_path,
    n_ctx=4096,  # The max sequence length to use - note that longer sequence lengths require much more resources
    n_threads=12, # Maximum I have is 12
    n_gpu_layers=-1, # The number of layers to offload to GPU, if you have GPU acceleration available.
    # verbose=False,
    flash_attn=True
)

llama_model_loader: loaded meta data with 25 key-value pairs and 339 tensors from C:/Users/benjc/Documents/models/Qwen2-7B-Instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = Qwen2-7B-Instruct
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 28
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_model_loader: - kv   4:                     qwen2.embedding_length u32              = 3584
llama_model_loader: - kv   5:                  qwen2.feed_forward_length u32              = 18944
llama_model_loader: - kv   6:                 qwen2.attention.head_count u32              = 28
llama_model_loader: - kv   7:           

In [205]:
def convert_to_qwen(original_sentence, prompt_input=final_prompt):
    """Function to convert the prompt into a Qwen formatted prompt"""

    # Qwen2 has the format
    
    # <|im_start|>system
    # {system_prompt}<|im_end|>
    # <|im_start|>user
    # {prompt}<|im_end|>
    # <|im_start|>assistant

    # Get the messages part of the prompt and initialise an empty string
    messages = prompt_input.messages
    formatted_messages = ""

    # For each item follow certain rules. If few shot example loop through all examples and append the 
    # input and output to the string as you do.
    for message in messages:
        
        if isinstance(message, SystemMessagePromptTemplate):
            formatted_messages += f"<|im_start|>system\n{message.prompt.template.replace('\n', '')}<|im_end|>\n"  
        elif isinstance(message, FewShotChatMessagePromptTemplate):
            for i in range(0, len(message.examples)):
                
                formatted_messages += f"<|im_start|>user\n{message.examples[i]['sentence'].replace('\n', '')}<|im_end|>\n"
                formatted_messages += f"<|im_start|>assistant\n{message.examples[i]['rephrased_sentences']}<|im_end|>\n"
                
        elif isinstance(message, HumanMessagePromptTemplate):
            formatted_messages += f"<|im_start|>user\n{message.prompt.template.replace('\n', '')}<|im_end|>\n"

    # Append the final tag for the beginning of the assistant message
    formatted_messages += f"<|im_start|>assistant"

    # I'm not using a chain so just replace the tag for the users' sentence with the actual sentence.
    formatted_prompt = formatted_messages.replace("<|im_start|>user\n{sentence}<|im_end|>",
                                                  f"<|im_start|>user\n{original_sentence}<|im_end|>")
    
    return formatted_prompt
    

In [206]:
original_sentence = "Usain Bolt, winner of the 100m sprint, was known for his lightning bolt celebration."

In [207]:
formatted_prompt = convert_to_qwen(original_sentence, prompt_input=final_prompt)

In [208]:
formatted_prompt

'<|im_start|>system\nGiven the sentence, generate at least twenty paraphrased versions that maintain the original semantic meaning and style. Return the rephrased sentences in a Python list format, ensuring the output starts with \'[\' and ends with \']\'. Include no additional text or notes in the output.<|im_end|>\n<|im_start|>user\nDBC Pierre, Booker Prize-winner and author of Vernon God Little, has moved to Ireland.<|im_end|>\n<|im_start|>assistant\n[\'Booker Prize-winner DBC Pierre, known for writing Vernon God Little, has moved to Ireland.\', \'The author of Vernon God Little and Booker Prize-winner, DBC Pierre, has settled in Ireland.\', \'DBC Pierre, who won the Booker Prize for Vernon God Little, has now moved to Ireland.\', \'DBC Pierre, celebrated for his Booker Prize-winning novel Vernon God Little, has moved to Ireland.\', \'Winner of the Booker Prize and author of Vernon God Little, DBC Pierre, has relocated to Ireland.\', \'The Booker Prize-winning author of Vernon God L

In [209]:
output_str = llm(formatted_prompt, max_tokens=1000, stop=["<|endoftext|>"], temperature=1)


llama_print_timings:        load time =   30753.65 ms
llama_print_timings:      sample time =     468.33 ms /   431 runs   (    1.09 ms per token,   920.28 tokens per second)
llama_print_timings: prompt eval time =   55418.88 ms /   849 tokens (   65.28 ms per token,    15.32 tokens per second)
llama_print_timings:        eval time =   91709.76 ms /   430 runs   (  213.28 ms per token,     4.69 tokens per second)
llama_print_timings:       total time =  153647.48 ms /  1279 tokens


In [210]:
output_str

{'id': 'cmpl-43361007-c0d5-4abd-b73d-3f1d5a8d1de9',
 'object': 'text_completion',
 'created': 1717718999,
 'model': 'C:/Users/benjc/Documents/models/Qwen2-7B-Instruct-Q4_K_M.gguf',
 'choices': [{'text': '\n[\'Usain Bolt, the champion in the 100-meter dash, was famous for executing the lightning bolt pose.\', \'The title holder of the 100m sprint, Usain Bolt, was renowned for his lightning bolt gesture.\', \'Usain Bolt, who triumphed in the 100m sprint, was recognized for his distinctive lighting bolt celebration.\', "Usain Bolt\'s victory in the 100m race was characterized by his iconic lightning bolt salute.", \'The person victorious in the 100-meter dash, Usain Bolt, was known for his signature lighting bolt dance move.\', "For being known for his lighting bolt pose, Usain Bolt won the 100 meter sprint.", \'Usain Bolt, the winner of the 100m race, is renowned for his lightning bolt gesture.\', \'The champion in the 100m sprint, Usain Bolt, was distinguished by his lightning bolt cele

In [211]:
output_text = output_str['choices'][0]['text']

In [212]:
print(output_text)


['Usain Bolt, the champion in the 100-meter dash, was famous for executing the lightning bolt pose.', 'The title holder of the 100m sprint, Usain Bolt, was renowned for his lightning bolt gesture.', 'Usain Bolt, who triumphed in the 100m sprint, was recognized for his distinctive lighting bolt celebration.', "Usain Bolt's victory in the 100m race was characterized by his iconic lightning bolt salute.", 'The person victorious in the 100-meter dash, Usain Bolt, was known for his signature lighting bolt dance move.', "For being known for his lighting bolt pose, Usain Bolt won the 100 meter sprint.", 'Usain Bolt, the winner of the 100m race, is renowned for his lightning bolt gesture.', 'The champion in the 100m sprint, Usain Bolt, was distinguished by his lightning bolt celebration.', 'Winning the 100m sprint, Usain Bolt was noted for his lighting bolt salute.', 'Usain Bolt, the winner of the 100-meter race, was famous for his lighting bolt pose.', "For winning the 100 meters sprint, Usa